# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [17]:
# number of callbacks for black-sounding names
print('Number of black sounding names called to interview :', sum(data[data.race=='b'].call))
print('Number of white sounding names called to interview :', sum(data[data.race=='w'].call))

Number of black sounding names called to interview : 157.0
Number of white sounding names called to interview : 235.0


In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

The research case we are looking for is a binomial situation. b and w labels assigned randomly to the cases, each call is independent from the other. When the n is large enough, binomial distribution approximates to the normal distribution and so calculations will be more simpler and easier. The assumption on this n large enough is

np >= 10
nq >= 10

in our case, white call rate*n > 10, black call rate*n > 10 , white not call rate*n > 10, black not call rate*n > 10

sample is large enough, samples are independent, CLT applicable.

In [32]:
w = data[data.race=='w']
b = data[data.race=='b']

print('white calls')
print(w.call.describe())
print('black calls')
print(b.call.describe())
print('white unique values')
print(w.call.unique())
print('black unique values')
print(b.call.unique())
print('white call count')
print(np.sum(w.call))
print('black call count')
print(np.sum(b.call))

w_rat = np.mean(w.call)
b_rat = np.mean(b.call)
print('white call ratio :', w_rat)
print('black call ratio :', b_rat)


print('general white call ratio')
w_ratgen = np.sum(w.call)/len(data.call)
print(w_ratgen)

print('general black call ratio')

b_ratgen = np.sum(b.call)/len(data.call) 
print(b_ratgen)


# since the call is a dichotom variable with unique values 0 and 1, mean is also ratio
# Is there any differences between white call ratio and black call ratio?
# H0 : white call ratio equals to black call ratio
# H1 : white call ratio is not equal to black call ratio

white calls
count    2435.000000
mean        0.096509
std         0.295346
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64
black calls
count    2435.000000
mean        0.064476
std         0.245649
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64
white unique values
[0. 1.]
black unique values
[0. 1.]
white call count
235.0
black call count
157.0
white call ratio : 0.09650924056768417
black call ratio : 0.0644763857126236
general white call ratio
0.048254620123203286
general black call ratio
0.03223819301848049


In [40]:
t_score, p_value = stats.ttest_ind(w.call, b.call)
if p_value <= 0.05:
    print(round(p_value, 5), ' <= 0.05, so H0 is rejected. There is a statistically significant difference between call rates')
else:
          print(round(p_value, 5), ' > 0.05, H0 cant be rejected.')

4e-05  <= 0.05, so H0 is rejected. There is a statistically significant difference between call rates


In [45]:
#this can be also tested via permutation bootstrapping methods
def boost_perm(data1, data2, size=1):
   
    perm_boost = np.empty(size)
    permutation = np.empty(size)
    
    for i in range(size):
        
        permutation = np.random.permutation(np.concatenate((data1, data2)))
        perm_samp1 = permutation[:len(data1)]
        perm_samp2 = permutation[len(data1):]
        perm_boost[i] = np.mean(perm_samp1) - np.mean(perm_samp2)

    return perm_boost

test_perm = boost_perm(w.call, b.call, size=10000)

p_perm = np.sum(test_perm>=(np.mean(w.call)-np.mean(b.call)))/len(test_perm)

if p_perm <= 0.05:
    print(p_perm, "is smaller than 0.05 selected alpha value and H0 is rejected. White and Black interview call rates are not equal.")
else:
    print(p_perm, "is not smaller than selected alpha, H0 can't be rejected.")


0.0 is smaller than 0.05 selected alpha value and H0 is rejected. White and Black interview call rates are not equal.


In [48]:
w_sem = stats.sem(w.call)
b_sem = stats.sem(b.call)

print('white SEM : ', w_sem)
print('black SEM : ', b_sem)


white SEM :  0.005985301397503016
black SEM :  0.0049781434352911685


In [50]:
print('white call CI :', stats.bayes_mvs(w.call))
print('black call CI :', stats.bayes_mvs(b.call))

white call CI : (Mean(statistic=0.09650924056768417, minmax=(0.08666631755160376, 0.10635216358376459)), Variance(statistic=0.08719521015882492, minmax=(0.08308479772540164, 0.0913056225922482)), Std_dev(statistic=0.2952883508688159, minmax=(0.28832835325744827, 0.3022483484801835)))
black call CI : (Mean(statistic=0.0644763857126236, minmax=(0.05628974971854314, 0.07266302170670405)), Variance(statistic=0.06031917780637741, minmax=(0.05747571085472273, 0.0631626447580321)), Std_dev(statistic=0.24559962908436447, minmax=(0.23981080325784432, 0.25138845491088463)))


Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.



Although our anaylsis can show the race effect on interview calls, race is not the only variable to measure, as can be seen in the data set there are 65 columns about the resumes, positions, education, age, gender, etc. These variables definitely has some effect on the calling to the interview, in order for more scientific and accurate call rate prediction regression models should be apllied.